In [18]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler

# ===== 1. Cargar dataset =====
df = pd.read_csv('Actividad 1. dataset_vivero.csv', sep=';', encoding='latin1')

print("\n Resumen inicial:")
print(df.info())
print(df.describe(include='all'))


 Resumen inicial:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   PlantaID   102 non-null    int64  
 1   Tipo       97 non-null     object 
 2   Altura_cm  97 non-null     object 
 3   Ancho_cm   102 non-null    object 
 4   Precio     98 non-null     object 
 5   En_stock   85 non-null     float64
dtypes: float64(1), int64(1), object(4)
memory usage: 4.9+ KB
None
          PlantaID      Tipo Altura_cm Ancho_cm Precio   En_stock
count   102.000000        97        97      102     98  85.000000
unique         NaN         4        90       92     83        NaN
top            NaN  Orquídea      81,5     24,7   12,0        NaN
freq           NaN        29         3        2      3        NaN
mean     49.627451       NaN       NaN      NaN    NaN   0.494118
std      29.385765       NaN       NaN      NaN    NaN   0.502933
min       1.000000       NaN

In [19]:
# ===== 2. Detectar problemas =====
print("\n🔍 Valores faltantes por columna:")
print(df.isnull().sum())

print(f"\n🔍 Filas duplicadas: {df.duplicated().sum()}")


🔍 Valores faltantes por columna:
PlantaID      0
Tipo          5
Altura_cm     5
Ancho_cm      0
Precio        4
En_stock     17
dtype: int64

🔍 Filas duplicadas: 2


In [20]:
# Columnas numéricas y categóricas
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print("\n Variables categóricas detectadas:", cat_cols)
print(" Variables numéricas detectadas:", num_cols)


 Variables categóricas detectadas: ['Tipo', 'Altura_cm', 'Ancho_cm', 'Precio']
 Variables numéricas detectadas: ['PlantaID', 'En_stock']


In [21]:
# Escalas distintas: mostrar rangos
print("\n Rangos de variables numéricas (posible diferencia de escala):")
for col in num_cols:
    print(f"{col}: min={df[col].min()}, max={df[col].max()}")



 Rangos de variables numéricas (posible diferencia de escala):
PlantaID: min=1, max=100
En_stock: min=0.0, max=1.0


In [22]:
# ===== 3. Imputación de valores faltantes =====
num_imputer = SimpleImputer(strategy='mean')  # Cambiar a 'median' si lo prefieres
df[num_cols] = num_imputer.fit_transform(df[num_cols])

cat_imputer = SimpleImputer(strategy='most_frequent')
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

In [23]:
# ===== 4. Codificación de variables categóricas =====
df_encoded = df.copy()
for col in cat_cols:
    if df[col].nunique() <= 2:
        # LabelEncoder para variables binarias
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df[col])
    else:
        # OneHotEncoder para variables con más categorías
        df_encoded = pd.get_dummies(df_encoded, columns=[col])

In [24]:
# ===== 5. Escalamiento =====
scaler = StandardScaler()  # Cambiar a MinMaxScaler() si lo prefieres
df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])

In [25]:
# ===== 6. Resultados =====
print("\n Datos originales (primeras filas):")
print(df.head())


 Datos originales (primeras filas):
   PlantaID      Tipo Altura_cm Ancho_cm Precio  En_stock
0       1.0      Rosa      18,3     42,1    4,7  0.000000
1       2.0  Orquídea      81,8     14,2   20,0  0.494118
2       3.0   Tulipán      48,0     18,1   20,3  1.000000
3       4.0   Tulipán      68,4     54,9   23,4  1.000000
4       5.0  Orquídea     110,3     40,3   26,2  0.000000


In [26]:
print("\n Datos procesados (primeras filas):")
print(df_encoded.head())


 Datos procesados (primeras filas):
   PlantaID      En_stock  Tipo_Girasol  Tipo_Orquídea  Tipo_Rosa  \
0 -1.662968 -1.082632e+00         False          False       True   
1 -1.628770  1.216273e-16         False           True      False   
2 -1.594572  1.108409e+00         False          False      False   
3 -1.560374  1.108409e+00         False          False      False   
4 -1.526175 -1.082632e+00         False           True      False   

   Tipo_Tulipán  Altura_cm_100,8  Altura_cm_100,9  Altura_cm_102,7  \
0         False            False            False            False   
1         False            False            False            False   
2          True            False            False            False   
3          True            False            False            False   
4         False            False            False            False   

   Altura_cm_105,4  ...  Precio_5,5  Precio_6,0  Precio_6,1  Precio_6,7  \
0            False  ...       False       False     

In [27]:
print("\n Información final:")
print(df_encoded.info())


 Información final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Columns: 271 entries, PlantaID to Precio_9,3
dtypes: bool(269), float64(2)
memory usage: 28.5 KB
None
